In [1]:
import pandas as pd
import os

import scipy.stats
from scipy.stats import hypergeom
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt

%pylab inline

import numpy
#import statsmodels.stats.multitest as smm
#import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [2]:
#import brain atlas data
ba=pd.read_csv("../data/BrainAtlas/Binary Brain atlas V17_w_120619_HB_data_Jan_10_2020.csv",sep=",")

#import signle cell data (see ipynb for converting data to binary)
sc=pd.read_csv("../data/scRNAseq_Cengen2020/071020_sc_neuron_class_vs_all_DE_WX_binary.csv",sep=',')

#imort curated list of motifs to use (excludes extra unc-3, che-1 motifs, removes repressors mab-9 and cog-1)
ctab=pd.read_csv("../data/motif_information/curated_motifs_071320b_with_neuron_class_expression_per_dataset.csv",sep=',')
ctab=ctab.loc[ctab['exclusion_rule']=='None']
#targetortho data
tfiles=os.listdir("../data/TargetOrtho_data/TargetOrtho_data_genes_corrected/")

#current gene ids (make sure using gene public name where available and transcript name if no gene name)
gids=pd.read_csv("../data/c_elegans.canonical_bioproject.current.geneIDs.txt",sep=',',header=None)
gids.columns=['index','WB_GID','gene_name','transcript_name']+list(gids.columns.values)[4:]
#import genenetic data (putative and bonefide terminal seelctors with at least one genetic markers tested)
#gdata=pd.read_csv("../data/genetic_data/090519_genetic_evidence_by_TF.csv",sep=',')

In [44]:
'jun-1' in ctab["TF_Name"].drop_duplicates().values

True

In [5]:
len(get_sc_genes('BDU'))

259

In [3]:
def get_sc_genes(neuron_class):
    return sorted(list(sc.loc[sc[neuron_class]==1]["gene_name"].values))

    

def get_ba_genes(neuron_class):
    df=ba.loc[ba["class"]==neuron_class]
    
    df=df.transpose()[4:]
    df.columns=list(range(len(df.columns)))
    genes=[]
    for i in range(len(df.columns)):
        g=list(df.index[df[i]==1])
        genes=genes+g
    #if gene is in isoform notation, reduce to main gene name (targetortho doesn't distinguish isoforms)
    correct_these={'inx-1a':'inx-1','inx-1b':'inx-1','inx-18a':'inx-18','inx-18b':'inx-18','inx-10a':'inx-10','inx-10b':'inx-10'}
    genes_corrected=[]
    for gene in genes:
        if gene not in correct_these.keys():
            genes_corrected.append(gene)
        else:
            genes_corrected.append(correct_these[gene])
    genes=list(set(genes_corrected))
    return sorted(list(set(genes)))

In [4]:
def import_TO_data(filename,motif_ID):
    tdata=pd.read_csv("../data/TargetOrtho_data/TargetOrtho_data_genes_corrected/%s" %(filename),sep=',')
    tdata["motif_name"]=motif_ID
    return tdata

def get_TO_df(TF,motif_ID): 
    tdata_df=pd.DataFrame()
    for tfile in tfiles:
        if tfile.startswith("%s_%s" %(TF,motif_ID)):
            tdata_df=tdata_df.append(import_TO_data(tfile,motif_ID))
    tdata_df=tdata_df[["ref_gene_name","Rank","motif_name"]]
    #ref_gene_names_corrected=check_gene_names(list(tdata_df['ref_gene_name'].values))
    #tdata_df['ref_gene_name']=ref_gene_names_corrected
    return tdata_df


"\ndef check_gene_names(ref_gene_names):\n    glist=[]\n    for g in ref_gene_names:\n        new_id=g\n        if g not in gids['gene_name'].values:\n            #check if gene name available by tid\n            gid=gids.loc[gids['transcript_name']==g]['gene_name'].values\n            if len(gid)>0:\n                if type(gid[0])!=float:\n                    #print(g,gid[0])\n                    new_id=gid[0]\n        glist.append(new_id)    \n    return glist\n"

In [5]:
def hypergeom_test(k,N,n):
    #total coding genes in genome
    M=20094.0
    #k=number of sucesses
    #k=number of BA genes in target ortho results.
    #k=len(BA_genes)
    #N=Brain Atlas neuron class marker count
    #N=len(ASE_genes_BA)
    #n=taragetortho data length (whole genome motif matchs per gene)
    #n=float(len(d))

    pval = hypergeom.sf(k-1, M, n, N)
    expected=N/M*n
    observed=k
    fold_enrichment=observed/expected
    prop_found=k/N
    return [observed,expected,fold_enrichment, pval]

def ranksums_test(gene_list, tdata):
    results=tdata.loc[tdata["ref_gene_name"].isin(gene_list)][["ref_gene_name","Rank"]]
    ranks_gene_list=results["Rank"]
    ranks_all_genes=tdata["Rank"]
    rs=scipy.stats.ranksums(ranks_gene_list,ranks_all_genes)
    return [rs[0],rs[1]]

def getGenes_w_motifs(gene_list,tdata):
    r=tdata.loc[tdata["ref_gene_name"].isin(gene_list)][["ref_gene_name","Rank"]]
    r=r.sort_values(ascending=True,by="Rank")
    return(r)

def multiple_test_correction(pval_col,df):
    FDR=.05
    pvals=list(df[pval_col].values)
    print(pval_col,'pvals',pvals[:10])
    corrected_ps=smm.multipletests(pvals, alpha=FDR, method='fdr_bh', is_sorted=False, returnsorted=False)
    print(corrected_ps[1][:10])
    return corrected_ps[1]

In [6]:
def assign_model(p1,p2,p3):
    c=.05
    #enriched and ranked
    if p1<c and p2 < c: mod='A-I'
    
    #enriched and not ranked sig. but has some subset.
    elif p1<c and p2 >= c and p3 < c: mod='A-II'
            
    #not enriched, yes ranked sig. 
    elif p1 >= c and p2 <c and p3 < c: mod='B-I'
        
    #not enriched, not ranked sig, yes subset ranked sig
    elif p1 >= c and p2 >=c and p3 < c: mod='B-II'
        
    #enriched but not rank sig. overall or in subset
    elif p1 < c and p2 >=c and p3 >= c: mod='C-I'

    #nothing sig.
    elif p1 >= c and p2 >=c and p3 >= c: mod='C-II'
    else:
        mod='error'
    return mod

In [7]:
def correct_pvals(df):
    #add multiple testing correctings for all pvals.
    pval_cols=["hg_pval","rs_pval","best_subset_rs_pval"]
    for pval_col in pval_cols:
        corrected_pvals=multiple_test_correction(pval_col,df)
        #print('corrected',corrected_pvals[:10])
        df["%s*" %pval_col]=corrected_pvals
    return df

In [8]:
def label_models(df):
    pval_cols=["hg_pval*","rs_pval*","best_subset_rs_pval*"]
    mods_list=[]
    for p1,p2,p3 in df[pval_cols].values:
        mod=assign_model(p1,p2,p3)
        mods_list.append(mod)
    df["model"]=mods_list
    return df

In [9]:
def subset_rs(genes_sorted_by_rank,tdata):
    
    if len(genes_sorted_by_rank)>0:
        j=len(genes_sorted_by_rank)
        plist=[]
        jlist=[]
        for i in range(len(genes_sorted_by_rank)):
            rs=ranksums_test(genes_sorted_by_rank[:j],tdata)
            #instead of returning largest subset, need to get best subset p value, otherwise mult test. correction. shows not. sig. even though a smaller subset may have been sig. 
            if rs[1]<.05:
                plist.append(rs[1])
                jlist.append(j)
                
            j-=1
        if len(plist)>0:
            min_p=min(plist)
            p_index=plist.index(min_p)
            j_val=jlist[p_index]
            return min_p, j_val
        else:
            return 1,0
    else:
        return 1,0
            

In [10]:
sc_neuron_profiles=sc.columns.values[2:]
#ignore DA_DB and unnanotated clusters
sc_neuron_profiles=set(sc_neuron_profiles)-set(['DA_DB','Unannotated'])
#VD and DD are clustered together, so expect 117 clusters covering 118 neuron classes
print(len(sc_neuron_profiles))
sc_use=sorted(list(set(sc_neuron_profiles)))

117


In [21]:
neuron_classes=sc_use
count=0
for TF, motif_ID in list(ctab[["TF_Name","motif_ID"]].drop_duplicates().values)[:]:
    present=False
    neurons=[]
    for neuron_class in  neuron_classes:
        sc_genes=get_sc_genes(neuron_class)
        ba_genes=get_ba_genes(neuron_class)
        if TF in sc_genes:
            present=True
            neurons.append(neuron_class)

        if TF in ba_genes:
            present=True
            neurons.append(neuron_class)
    print(count, TF, motif_ID, neurons)
    count+=1



0 cfi-1 M01667_2.00 ['AVD', 'AVD', 'DA', 'IL2', 'IL2', 'LUA', 'PVC', 'PVC', 'RIP', 'URA', 'URA']
1 lin-15B M09654_2.0 ['ADF', 'AIN', 'AIY', 'CAN', 'DA', 'DB', 'DVB', 'HSN', 'I4', 'M2', 'NSM', 'VA', 'VB', 'VD_DD']
2 mdl-1 M08183_2.00 ['ADF', 'ASG', 'ASH', 'ASI', 'ASK', 'AVG', 'AVJ', 'CAN', 'CEP', 'DVA', 'PHA', 'PHB', 'PVP', 'RIA', 'RIC', 'RIM', 'SIA', 'SMB', 'SMD']
3 hlh-30 M00937_2.00 ['ADF', 'ASK', 'AVD', 'AVJ', 'AVM', 'AWC', 'BDU', 'DA', 'HSN', 'PLM', 'PVT', 'PVW']
4 hlh-11 M00931_2.00 ['CAN', 'DB', 'I4', 'IL2', 'NSM', 'PDB']
5 mxl-3 M00935_2.00 ['ADF', 'ADL', 'ASE', 'AVL', 'AWC', 'AWC', 'SDQ']
6 hlh-10 M00930_2.00 ['ASE', 'RIC', 'RIG']
7 hlh-4 OH2018_HLH-4 ['ADL', 'ADL']
8 hlh-16 M00589_2.00 ['AIY', 'SIA', 'SIB', 'SMD']
9 xbp-1 M00597_2.00 ['AIZ', 'AVB', 'AVG', 'AVH', 'AVJ', 'DVB', 'DVC', 'M5', 'PLN', 'PVP', 'PVR', 'PVW', 'RID', 'RIF', 'RIV', 'RMF', 'SIA', 'SIB', 'SMB', 'SMD', 'VB']
10 atf-7 M00595_2.00 ['AFD', 'AIM', 'AIZ', 'ALM', 'ASG', 'AWA', 'AWC', 'PLM', 'PVW', 'RIA', 'RIG', 'V

40 ceh-34 M00672_2.00 ['I1', 'I1', 'I2', 'I2', 'I3', 'I3', 'I4', 'I4', 'I5', 'I5', 'I6', 'I6', 'M1', 'M1', 'M2', 'M2', 'M3', 'M3', 'M4', 'M4', 'M5', 'M5', 'MC', 'MC', 'MI', 'MI', 'NSM', 'NSM']
41 lin-39 M08211_2.00 ['AIY', 'AQR', 'AQR', 'AS', 'AS', 'AVM', 'AVM', 'DA', 'DA', 'DB', 'DB', 'PDE', 'PDE', 'PVD', 'PVD', 'SDQ', 'SDQ', 'VA', 'VA', 'VB', 'VC', 'VC']
42 ceh-32 M00483_2.00 ['ADL', 'I2', 'I2', 'IL1', 'M1', 'M1', 'OLL', 'OLL', 'RIA', 'RIA', 'RIB', 'RIB', 'RID', 'RID', 'RMD', 'RMD', 'RME', 'RME', 'RMH', 'SMD', 'URY', 'URY']
43 ceh-54 M02188_2.00 ['AFD', 'AFD', 'ASE', 'ASG', 'ASG', 'ASH', 'AUA', 'AWA', 'AWA', 'AWC', 'BAG', 'BAG', 'I2', 'M3', 'M3']
44 ceh-13 M00450_2.00 ['AS', 'DA', 'DA', 'DB', 'RMG', 'VA', 'VB', 'VC']
45 ceh-57 M00675_2.00 ['ADF', 'ADF', 'ADL', 'ASG', 'ASG', 'ASI', 'ASI', 'ASJ', 'ASJ', 'ASK']
46 ceh-43 M00423_2.00 ['ADE', 'ADE', 'AIZ', 'AIZ', 'ASJ', 'BDU', 'CAN', 'CEP', 'CEP', 'IL1', 'PDE', 'PVQ', 'SDQ', 'URB']
47 lim-4 M02182_2.00 ['AWB', 'AWB', 'RID', 'RID', 'RIV', 

117 dmd-8 M00660_2.00 ['AFD']
118 elt-1 M08206_2.00 ['AS', 'PDB', 'RIF', 'SAB']
119 egl-27 M08205_2.00 ['DA', 'DB', 'IL2', 'M1', 'RIG']
120 nhr-46 M00692_2.00 ['RID']
121 nhr-66 M00694_2.00 ['CAN']
122 nhr-84 M02414_2.00 ['ASG', 'ASK', 'CEP', 'I5', 'M1', 'M3', 'M5', 'MC', 'MI', 'NSM', 'PDE']
123 nhr-6 M01298_2.00 ['ADL', 'AFD', 'AIB', 'ALA', 'ASI', 'AVH', 'AVK', 'AWC', 'BAG', 'BDU', 'DVA', 'DVC', 'PVM', 'PVR', 'PVW', 'RID']
124 nhr-67 M02408_2.00 ['ADF', 'AIN', 'AIN', 'ASE', 'ASG', 'ASJ', 'AUA', 'AVA', 'AVG', 'AVL', 'AWB', 'CEP', 'CEP', 'DVA', 'PVP', 'RIH', 'RIR', 'RIR', 'RIS', 'RIS', 'RME', 'RME', 'SIA']
125 nhr-36 M02418_2.00 ['ADF', 'ADL', 'AFD', 'AFD', 'ASE', 'ASH', 'ASJ', 'AWC', 'AWC']
126 fax-1 M06432_2.00 ['AIY', 'AVA', 'AVA', 'AVB', 'AVB', 'AVE', 'AVE', 'AVK', 'AVK', 'CEP', 'DVA', 'DVA', 'I3', 'M1', 'M4', 'M5', 'MI', 'RIC', 'RIC', 'SAB', 'SIB', 'SIB', 'URX']
127 nhr-236 M06431_2.00 ['M2', 'MC', 'RIP']
128 nhr-216 M00705_2.00 ['AWA']
129 nhr-239 M06434_2.00 ['NSM']
130 odr-7 M00

In [48]:

def run(data_source):
    headers="TF,motif_ID,neuron_class,marker_count,obs_motif,exp_motif,fc,hg_pval,rs_pval,best_subset_rs_pval,num_genes_in_subset".split(',')
    df=pd.DataFrame()
    #get neuron_classes
    if data_source=='sc':
        neuron_classes=sc_use
   
    elif data_source=='ba':
        neuron_classes=sorted(list(ba["class"].drop_duplicates().values))
    
    for TF, motif_ID in list(ctab[["TF_Name","motif_ID"]].drop_duplicates().values)[:]:
        
        #TF='ceh-10--ttx-3' 
        #motif_ID='OH2007_CEH-10_TTX-3'
        #only need dto fetch targetortho data one time per motif_ID.
        print(TF, motif_ID)
        
        
        tdata=get_TO_df(TF,motif_ID)
        for neuron_class in  neuron_classes:
            
            present=False
            
            #check if TF is expressed in theshold 2 data or BA
            #motif file info (ctab) has all neuron classes exprsesed per TF
            
            #th2_expressed_cells=ctab.loc[ctab["TF_Name"]==TF]["single cell DE data: neuron classes that express (threshold 2)"].values
            #if len(th2_expressed_cells)>0:
            #    print(th2_expressed_cells)
            
            
            
            #get ba genes for neuron class
            if neuron_class=="VD_DD":
                genes_ba=list(set(get_ba_genes("VD")+get_ba_genes("DD")))
            else:
                genes_ba=get_ba_genes(neuron_class)
            
            
            
            
            #get sc genes for neuron class
            if neuron_class in ["DD","VD"]:
                genes_sc=get_sc_genes("VD_DD")
            else:
                genes_sc=get_sc_genes(neuron_class)
            
            if TF in genes_sc:
                present=True
            if TF in genes_ba:
                present=True
             
            if TF=='ceh-10--ttx-3':
                if 'ceh-10' in genes_sc:
                    if 'ttx-3' in genes_sc:
                        present=True
                if 'ceh-10' in genes_ba:
                    if 'ttx-3' in genes_ba:
                        present=True
            
            
            #deal with ceh-10--ttx-3, VD_DD, and jun-1 (1-jun)
            if data_source=='sc':
                genes=genes_sc
            else:
                genes=genes_ba
                
            if present==True and len(genes)<20:
                print('fewer than 20 genes', len(genes_sc), len(genes_ba))

            if present==True and len(genes)>=20:
                print(present,len(genes), neuron_class)  
                #run statistical tests on motif enrichment and ranking
                
                #run rank sums test on neuron class gene battery (are targetortho ranks of these genes sig. better than overall ranks for this motif)
                rs=ranksums_test(genes,tdata)
                rs_pval=rs[1]

                #do rank sums test on subsets of genes to find most sig. subset.
                r=getGenes_w_motifs(genes,tdata)
                genes_sorted_by_rank=r["ref_gene_name"].values
                best_pval,num_genes_in_subset=subset_rs(genes_sorted_by_rank,tdata)

                #run hypergeometric test for enrichment
                hg=hypergeom_test(len(genes_sorted_by_rank),len(genes),len(tdata))
                obs=hg[0]
                exp=hg[1]
                fc=hg[2]
                hg_pval=hg[3]

                results=[TF, motif_ID, neuron_class,len(genes), obs,exp,fc,hg_pval, rs_pval, best_pval, num_genes_in_subset]
                print(results)

                #add results to dataframe
                dictionary = dict(zip(headers, results))
                df=df.append(dictionary,ignore_index=True)
                
        #if present==False:
        #    print(TF, motif_ID)
    #run multiple testing correction on unqiue dataset and categorize into models
    df=df.drop_duplicates()
    df=correct_pvals(df)
    df=label_models(df)
    cols=['model']+headers+["hg_pval*","rs_pval*","best_subset_rs_pval*"]
    return df[cols]
    

In [49]:
data_source='sc'
df=run(data_source)
df.to_csv("../output/072020_single_factor_analysis_data_DE_NB_sc.csv",sep=',')

cfi-1 M01667_2.00
True 256 AVD
['cfi-1', 'M01667_2.00', 'AVD', 256, 201, 148.01473076540262, 1.3579729460750558, 1.8188016030274443e-12, 1.5866637399756668e-21, 1.5017927282876085e-43, 123]
True 901 DA
['cfi-1', 'M01667_2.00', 'DA', 901, 692, 520.9424703891709, 1.3283616509190357, 2.062971855936051e-34, 2.2268913469408045e-56, 4.052669626680072e-130, 414]
True 735 IL2
['cfi-1', 'M01667_2.00', 'IL2', 735, 462, 424.96416840848013, 1.0871504807810541, 0.0025967927006552384, 9.470962328480703e-13, 3.9405581352166767e-69, 223]
True 71 LUA
['cfi-1', 'M01667_2.00', 'LUA', 71, 59, 41.05096048571713, 1.4372379915575395, 4.8982743361416335e-06, 0.002337840870948605, 4.5278392894915454e-11, 27]
True 427 PVC
['cfi-1', 'M01667_2.00', 'PVC', 427, 307, 246.88394545635515, 1.243499245900833, 7.965724708081586e-10, 6.829029975642677e-32, 1.3205039129692191e-69, 189]
True 134 RIP
['cfi-1', 'M01667_2.00', 'RIP', 134, 110, 77.47646063501543, 1.419786075646899, 1.7294825449482272e-09, 6.028213232747892e-10

['hlh-30', 'M00937_2.00', 'PLM', 341, 138, 78.97949636707476, 1.7472889338090274, 5.376121465192418e-13, 0.008354798495843115, 6.4530216366796946e-21, 63]
True 193 PVT
['hlh-30', 'M00937_2.00', 'PVT', 193, 92, 44.701005275206526, 2.0581192622759183, 6.33691848067838e-14, 0.0014041623257193669, 3.987152230492381e-17, 43]
True 570 PVW
['hlh-30', 'M00937_2.00', 'PVW', 570, 220, 132.01851298895193, 1.6664329495849637, 4.173705202667173e-17, 2.8274975925102213e-05, 1.880080349537253e-34, 110]
hlh-11 M00931_2.00
True 1166 CAN
['hlh-11', 'M00931_2.00', 'CAN', 1166, 320, 369.0534487906838, 0.867083077122237, 0.9994242826043981, 0.5166710718973605, 1.27792381120181e-35, 118]
True 1280 DB
['hlh-11', 'M00931_2.00', 'DB', 1280, 594, 405.13586145117944, 1.4661748231132077, 3.1604167785620126e-30, 0.14859121431553843, 2.7023141001048896e-76, 257]
True 698 I4
['hlh-11', 'M00931_2.00', 'I4', 698, 225, 220.9256494475963, 1.0184421798129426, 0.3819616288834601, 0.9433338435820087, 9.833319237993762e-28,

['atf-7', 'M00595_2.00', 'AWA', 386, 195, 130.6836866726386, 1.4921525782209768, 7.594698063913465e-12, 7.64391912003423e-06, 2.2703305581787502e-29, 89]
True 433 AWC
['atf-7', 'M00595_2.00', 'AWC', 433, 226, 146.5959490395143, 1.541652422735656, 1.7605549539740951e-15, 1.125498188002621e-10, 2.338289106416521e-40, 116]
True 341 PLM
['atf-7', 'M00595_2.00', 'PLM', 341, 204, 115.44854185328954, 1.7670210184139048, 4.632086298320289e-23, 7.104374362665406e-13, 4.374087371756636e-37, 115]
True 570 PVW
['atf-7', 'M00595_2.00', 'PVW', 570, 327, 192.9785010450881, 1.6944892733082153, 1.9901699183947637e-31, 4.99715236779565e-28, 3.012905624750431e-69, 186]
True 440 RIA
['atf-7', 'M00595_2.00', 'RIA', 440, 179, 148.96586045585747, 1.2016176018601419, 0.001479400100755862, 5.879983178285433e-12, 3.982297761937137e-34, 100]
True 846 RIG
['atf-7', 'M00595_2.00', 'RIG', 846, 429, 286.42072260376233, 1.4977966541669663, 4.973168420599985e-25, 5.191244061046513e-37, 2.395033018933428e-89, 256]
True

['crh-1', 'M03659_2.00', 'ASE', 336, 136, 85.6470588235294, 1.587912087912088, 9.696282409990147e-10, 0.7144873779114651, 9.624175293023161e-17, 50]
True 436 ASJ
['crh-1', 'M03659_2.00', 'ASJ', 436, 169, 111.13725490196079, 1.5206422018348624, 5.117768750951104e-10, 0.5781823830024693, 5.628221111447544e-18, 59]
True 259 BDU
['crh-1', 'M03659_2.00', 'BDU', 259, 153, 66.01960784313725, 2.3174933174933177, 1.3084377096583229e-30, 0.0040982517153845925, 1.9800943592202944e-27, 64]
True 698 HSN
['crh-1', 'M03659_2.00', 'HSN', 698, 272, 177.921568627451, 1.528763500110205, 1.1732407505320746e-15, 0.03383894896487892, 6.812509706356615e-35, 107]
True 953 NSM
['crh-1', 'M03659_2.00', 'NSM', 953, 286, 242.92156862745097, 1.1773347324239245, 0.0007019520274507544, 0.2685768524529568, 3.010814540682202e-37, 109]
True 846 RIG
['crh-1', 'M03659_2.00', 'RIG', 846, 347, 215.64705882352942, 1.6091107474086197, 4.558499009057974e-24, 0.0031121231760921665, 2.758889147957984e-51, 141]
skn-1 M00596_2.00

['ceh-38', 'M03716_2.00', 'ADA', 273, 153, 133.48387578381607, 1.1462058552134886, 0.010189788043291702, 0.008028499952511974, 9.117572697236218e-21, 61]
True 260 ADE
['ceh-38', 'M03716_2.00', 'ADE', 260, 143, 127.12750074649148, 1.1248549618320611, 0.027425444582709846, 0.020820153983226414, 2.1029342641676447e-20, 59]
True 525 ADF
['ceh-38', 'M03716_2.00', 'ADF', 525, 274, 256.6997611227232, 1.0673948382406397, 0.06860707686014361, 0.007834736709422967, 6.81794629446674e-34, 103]
True 595 ADL
['ceh-38', 'M03716_2.00', 'ADL', 595, 273, 290.9263959390863, 0.9383816793893129, 0.9375755264327252, 0.473653928369672, 4.0427948982037855e-25, 83]
True 370 AFD
['ceh-38', 'M03716_2.00', 'AFD', 370, 211, 180.91221260077634, 1.1663115329069529, 0.0009402054544585687, 0.00010290723752144406, 9.825742721309797e-34, 104]
True 89 AIA
['ceh-38', 'M03716_2.00', 'AIA', 89, 55, 43.51672140937593, 1.2638819795865854, 0.009629616685084042, 0.20685560380036583, 1.6776072768405814e-09, 26]
True 242 AIB
['ce

...

['ztf-11', 'M00650_2.00', 'AIB', 242, 189, 155.46819946252614, 1.215682696869184, 1.9168333918247176e-06, 2.2669433581445e-17, 2.358910724725862e-38, 115]
True 240 AIM
['ztf-11', 'M00650_2.00', 'AIM', 240, 209, 154.18333830994328, 1.3555290882330155, 9.349858289787257e-16, 3.4137045798284384e-24, 1.270330840305253e-46, 128]
True 157 AIN
['ztf-11', 'M00650_2.00', 'AIN', 157, 127, 100.86160047775456, 1.2591511476958008, 3.6111305271035798e-06, 1.3866066788798692e-12, 3.762267566360132e-30, 74]
True 177 AIY
['ztf-11', 'M00650_2.00', 'AIY', 177, 143, 113.71021200358315, 1.2575827401983375, 1.029469828425609e-06, 4.060958015689448e-07, 2.248265592814552e-27, 75]
True 1144 AIZ
['ztf-11', 'M00650_2.00', 'AIZ', 1144, 705, 734.9405792773962, 0.9592612244831628, 0.9729958811557287, 1.0196954215685206e-35, 5.535022984219541e-118, 356]
True 163 ALA
['ztf-11', 'M00650_2.00', 'ALA', 163, 128, 104.71618393550314, 1.2223516479442933, 5.2805390332035445e-05, 7.313018051283939e-12, 3.775014472828931e-25

['ztf-11', 'M00650_2.00', 'IL2', 735, 483, 472.1864735742012, 1.0229009661033832, 0.2097087011539631, 1.6731284621594666e-24, 3.3326855650903953e-79, 243]
True 71 LUA
['ztf-11', 'M00650_2.00', 'LUA', 71, 64, 45.61257091669155, 1.4031219620769, 5.918635018981415e-07, 1.1528147737601253e-09, 3.3347715699297823e-17, 41]
True 413 M1
['ztf-11', 'M00650_2.00', 'M1', 413, 316, 265.3238280083607, 1.1909974402677563, 3.955174908826907e-08, 3.1106562211523006e-26, 1.0166804034986172e-62, 187]
True 500 M2
['ztf-11', 'M00650_2.00', 'M2', 500, 279, 321.21528814571514, 0.8685763420869161, 0.9999659402479615, 7.176798218260442e-19, 1.7192874845996812e-53, 155]
True 128 M3
['ztf-11', 'M00650_2.00', 'M3', 128, 107, 82.23111376530308, 1.3012106379270276, 9.958362286258068e-07, 1.2313541400907054e-13, 1.4456452704825735e-23, 73]
True 52 M4
['ztf-11', 'M00650_2.00', 'M4', 52, 42, 33.406389967154375, 1.2572444984715494, 0.007449929385432995, 0.0005934233288639691, 3.0038322003927642e-09, 23]
True 307 M5
['

['ztf-11', 'M00650_2.00', 'SDQ', 98, 80, 62.95819647656017, 1.2706844299420907, 0.00012479803663926922, 3.845514220222973e-09, 8.819616584580749e-18, 50]
True 583 SIA
['ztf-11', 'M00650_2.00', 'SIA', 583, 350, 374.53702597790385, 0.9344870486066404, 0.9854465907654552, 7.154163762523105e-10, 4.638424835839059e-51, 158]
True 336 SIB
['ztf-11', 'M00650_2.00', 'SIB', 336, 208, 215.85667363392056, 0.9636023593727522, 0.8313945297787205, 9.8066924102069e-10, 9.257619589845714e-32, 100]
True 534 SMB
['ztf-11', 'M00650_2.00', 'SMB', 534, 377, 343.0579277396238, 1.098939769397015, 0.0009668676579787179, 2.4321010334587626e-26, 1.565574702947712e-70, 211]
True 402 SMD
['ztf-11', 'M00650_2.00', 'SMD', 402, 317, 258.257091669155, 1.2274590329782646, 9.5167899530174e-11, 1.1879713312113928e-41, 4.828128233918983e-69, 213]
True 170 URA
['ztf-11', 'M00650_2.00', 'URA', 170, 134, 109.21319796954315, 1.2269579363234953, 2.52777349141344e-05, 2.598657723648832e-19, 3.342526460001081e-33, 91]
True 271 U

['egl-27', 'M08205_2.00', 'RIG', 846, 454, 374.96148103911617, 1.2107910357667873, 1.5799411361929585e-08, 1.2774123754393945e-19, 6.332677955040495e-65, 219]
nhr-46 M00692_2.00
True 258 RID
['nhr-46', 'M00692_2.00', 'RID', 258, 146, 108.12272320095552, 1.3503174511119762, 1.2523056478023868e-06, 8.304961080938977e-10, 5.902264188304026e-26, 78]
nhr-66 M00694_2.00
True 1166 CAN
['nhr-66', 'M00694_2.00', 'CAN', 1166, 424, 495.43684681994625, 0.8558103877850891, 0.9999950987650361, 2.614563761353645e-05, 9.502129959914179e-54, 173]
nhr-84 M02414_2.00
True 558 ASG
['nhr-84', 'M02414_2.00', 'ASG', 558, 252, 207.93789190803227, 1.2119003308519436, 6.479437291633857e-05, 1.1802984793406933e-12, 3.080740746817905e-45, 118]
True 545 ASK
['nhr-84', 'M02414_2.00', 'ASK', 545, 276, 203.09346073454762, 1.3589802399435427, 7.894092779100763e-11, 4.920707250496945e-10, 1.7558519699203078e-42, 134]
True 291 CEP
['nhr-84', 'M02414_2.00', 'CEP', 291, 132, 108.44072857569424, 1.217254824213587, 0.002655

['nhr-36', 'M02418_2.00', 'ADL', 595, 221, 194.21742808798646, 1.137899941193126, 0.010356513606059652, 0.4414549364092607, 4.300511889931469e-25, 80]
True 370 AFD
['nhr-36', 'M02418_2.00', 'AFD', 370, 194, 120.77386284463023, 1.6063078171936231, 1.800273316477049e-15, 0.0005051566324405398, 4.496104623307628e-30, 84]
True 336 ASE
['nhr-36', 'M02418_2.00', 'ASE', 336, 166, 109.6757240967453, 1.513552806394703, 1.0424146350035405e-10, 0.001074836379889796, 9.97178812507429e-25, 68]
True 451 ASH
['nhr-36', 'M02418_2.00', 'ASH', 451, 193, 147.21354633223848, 1.3110206554254762, 3.2746478130329717e-06, 0.04818978480546259, 6.345908066859457e-23, 74]
True 436 ASJ
['nhr-36', 'M02418_2.00', 'ASJ', 436, 183, 142.31730864934806, 1.2858590549297764, 2.3223450619054577e-05, 0.4763931579461369, 4.423979376484977e-20, 63]
True 433 AWC
['nhr-36', 'M02418_2.00', 'AWC', 433, 206, 141.33806111277, 1.4574984146389125, 5.049344937748403e-11, 0.022125965065049626, 1.4938368050951546e-28, 78]
fax-1 M06432_

In [51]:
data_source='ba'
df=run(data_source)
df.to_csv("../output/072020_single_factor_analysis_data_DE_NB_ba.csv",sep=',')

cfi-1 M01667_2.00
True 66 AVD
['cfi-1', 'M01667_2.00', 'AVD', 66, 52, 38.16004777545536, 1.3626817321084836, 0.00027798857032293537, 0.0013978029565782611, 7.574767626279522e-09, 27]
True 78 DA
['cfi-1', 'M01667_2.00', 'DA', 78, 62, 45.098238280083606, 1.3747765403816359, 4.469079172573879e-05, 1.9824736295251144e-05, 2.7372425309763044e-13, 36]
True 62 IL2
['cfi-1', 'M01667_2.00', 'IL2', 62, 45, 35.84731760724595, 1.2553240522215248, 0.011612362443663758, 0.017641555177541495, 8.825696519128614e-08, 22]
True 50 LUA
['cfi-1', 'M01667_2.00', 'LUA', 50, 38, 28.9091271026177, 1.3144637631261835, 0.005742794560669289, 0.04103617514466043, 3.963404918865555e-07, 18]
True 80 PVC
['cfi-1', 'M01667_2.00', 'PVC', 80, 65, 46.25460336418831, 1.4052655362368738, 7.465707734213825e-06, 6.913352058650986e-07, 9.297892957600288e-14, 41]
True 36 RIP
['cfi-1', 'M01667_2.00', 'RIP', 36, 27, 20.81457151388474, 1.297168187295576, 0.025047749596048184, 0.13536736627381413, 0.00015487710719073766, 13]
True 

['hlh-30', 'M00937_2.00', 'PVT', 153, 34, 35.43654822335026, 0.959461395215585, 0.6396850989603525, 0.11245445741644239, 6.219932832982367e-06, 17]
True 40 PVW
['hlh-30', 'M00937_2.00', 'PVW', 40, 11, 9.264457051856276, 1.1873334765792867, 0.31233148530711063, 0.22615369567073673, 1, 0]
hlh-11 M00931_2.00
True 89 CAN
['hlh-11', 'M00931_2.00', 'CAN', 89, 48, 28.16960286652732, 1.7039643841424634, 1.0677562188326587e-05, 0.5076960032418023, 1.418900336310782e-07, 18]
True 63 DB
['hlh-11', 'M00931_2.00', 'DB', 63, 34, 19.94028068080024, 1.7050913447139862, 0.00019608470771232902, 0.694375380228254, 3.2618415869247293e-06, 14]
True 42 I4
['hlh-11', 'M00931_2.00', 'I4', 42, 22, 13.293520453866826, 1.654941599281222, 0.004166140871729964, 0.11980930366690379, 0.0025029014211661914, 7]
True 62 IL2
['hlh-11', 'M00931_2.00', 'IL2', 62, 33, 19.623768289041507, 1.6816342057212414, 0.00034058130932312995, 0.18808604815738983, 8.171679284243533e-07, 15]
True 78 NSM
['hlh-11', 'M00931_2.00', 'NSM', 

['atf-7', 'M00595_2.00', 'PLM', 133, 76, 45.02831691052055, 1.687826799101237, 2.919032018294354e-08, 5.872902844146657e-05, 5.300569568215287e-13, 42]
True 40 PVW
['atf-7', 'M00595_2.00', 'PVW', 40, 20, 13.542350950532496, 1.4768484492135823, 0.02532188210962134, 0.022391419945363852, 0.000163944003487356, 10]
True 56 RIA
['atf-7', 'M00595_2.00', 'RIA', 56, 28, 18.959291330745497, 1.4768484492135823, 0.009065390584134851, 5.8957045944201804e-05, 1.234335916434491e-07, 19]
True 58 RIG
['atf-7', 'M00595_2.00', 'RIG', 58, 29, 19.63640887827212, 1.4768484492135823, 0.007993249744421364, 0.003466593216751042, 7.329054846334506e-06, 17]
True 61 VB
['atf-7', 'M00595_2.00', 'VB', 61, 33, 20.65208519956206, 1.5979016007884659, 0.000896295708146975, 0.032639248003600185, 7.630248568033774e-06, 18]
cebp-1 M00604_2.00
True 54 AIZ
['cebp-1', 'M00604_2.00', 'AIZ', 54, 47, 35.49208719020603, 1.324238829576908, 0.00035796019241534944, 1.394275367533144e-07, 2.226510624426139e-12, 30]
True 55 DD
['ceb

True 68 BDU
['crh-1', 'M03659_2.00', 'BDU', 68, 31, 17.333333333333332, 1.7884615384615385, 0.00025655277536144704, 0.5856024551713717, 0.01000800370778963, 10]
True 122 HSN
['crh-1', 'M03659_2.00', 'HSN', 122, 62, 31.098039215686274, 1.9936948297604036, 1.6533143393933082e-09, 0.560938004569462, 1.7573280225314626e-08, 18]
True 78 NSM
['crh-1', 'M03659_2.00', 'NSM', 78, 41, 19.88235294117647, 2.062130177514793, 2.824969630063454e-07, 0.5738704814418308, 4.735876799186511e-06, 13]
True 58 RIG
['crh-1', 'M03659_2.00', 'RIG', 58, 21, 14.784313725490197, 1.4204244031830238, 0.045912169049726184, 0.32538062185354155, 0.0477900959203229, 9]
skn-1 M00596_2.00
True 254 ASI
['skn-1', 'M00596_2.00', 'ASI', 254, 109, 82.58096944361502, 1.3199166918768552, 0.00030782803398959977, 0.07401212839668955, 4.4083233556870374e-17, 46]
atf-5 M00593_2.00
True 114 ADF
['atf-5', 'M00593_2.00', 'ADF', 114, 59, 34.73783218871305, 1.698436438966107, 1.642342395474838e-06, 0.08823822764954577, 4.347332683898179

['ceh-38', 'M03716_2.00', 'ADL', 190, 106, 92.9008659301284, 1.1410012053033345, 0.033042028337421855, 0.3157287920066375, 1.0578108420001408e-14, 38]
True 96 AFD
['ceh-38', 'M03716_2.00', 'AFD', 96, 63, 46.939384891012246, 1.3421564885496182, 0.0006787925094791694, 0.0013035629249590719, 6.49931697308989e-12, 27]
True 53 AIA
['ceh-38', 'M03716_2.00', 'AIA', 53, 35, 25.91445207524634, 1.35059772432666, 0.008774626858917509, 0.0016463199091884817, 5.7191541174373004e-08, 19]
True 54 AIB
['ceh-38', 'M03716_2.00', 'AIB', 54, 36, 26.403404001194385, 1.3634605597964378, 0.006303883601855819, 0.000898577297076542, 8.230318663181553e-08, 18]
True 62 AIM
['ceh-38', 'M03716_2.00', 'AIM', 62, 42, 30.315019408778742, 1.385451859147993, 0.002069843046271993, 0.011784619788760175, 4.157883788879192e-08, 19]
True 50 AIN
['ceh-38', 'M03716_2.00', 'AIN', 50, 35, 24.44759629740221, 1.4316335877862596, 0.002019563116650286, 0.0655203612232901, 5.619634863027586e-06, 16]
True 108 AIY
['ceh-38', 'M03716_2

...

['ztf-11', 'M00650_2.00', 'AVK', 47, 37, 30.194237085697225, 1.2253993997260693, 0.024126803947213744, 6.773945061081325e-05, 5.8145832835090136e-08, 22]
True 41 AVL
['ztf-11', 'M00650_2.00', 'AVL', 41, 32, 26.33965362794864, 1.2148982842373162, 0.04255686221356683, 6.545406117246591e-05, 2.1740896579550907e-08, 17]
True 93 AVM
['ztf-11', 'M00650_2.00', 'AVM', 93, 75, 59.74604359510302, 1.2553132473218234, 0.000422352311193148, 3.651679858409223e-09, 1.3295094628144986e-16, 48]
True 85 AWA
['ztf-11', 'M00650_2.00', 'AWA', 85, 70, 54.60659898477157, 1.2818963513827561, 0.00019411388712280584, 2.1973628499202917e-06, 4.497066642595541e-14, 44]
True 119 AWB
['ztf-11', 'M00650_2.00', 'AWB', 119, 98, 76.44923857868021, 1.2818963513827561, 1.0687988686981351e-05, 6.1051048826105896e-06, 2.892432633818517e-17, 54]
True 161 AWC
['ztf-11', 'M00650_2.00', 'AWC', 161, 127, 103.43132278292028, 1.227867889367955, 3.824028365708346e-05, 1.3987636843779468e-10, 2.0699956817568247e-26, 77]
True 87 BAG

['ztf-11', 'M00650_2.00', 'PVP', 51, 43, 32.763959390862944, 1.3124176930823457, 0.0013244489665062572, 1.4547128916892125e-07, 1.0927009843288325e-11, 27]
True 83 PVQ
['ztf-11', 'M00650_2.00', 'PVQ', 83, 67, 53.32173783218871, 1.25652318780117, 0.0008064459977409303, 2.0316415297309957e-09, 3.9473917544492813e-16, 46]
True 54 PVR
['ztf-11', 'M00650_2.00', 'PVR', 54, 45, 34.69125111973723, 1.297157022232551, 0.0016891161174663793, 1.7109544546732433e-06, 4.486490317445535e-11, 25]
True 153 PVT
['ztf-11', 'M00650_2.00', 'PVT', 153, 105, 98.29187817258884, 1.06824695948563, 0.14635732957719255, 0.014121816552010466, 2.0347444944542384e-15, 45]
True 40 PVW
['ztf-11', 'M00650_2.00', 'PVW', 40, 32, 25.69722305165721, 1.2452707413432489, 0.024089725730796725, 0.008053326345047544, 2.683136659232143e-06, 18]
True 56 RIA
['ztf-11', 'M00650_2.00', 'RIA', 56, 44, 35.976112272320094, 1.2230337638192623, 0.015269083276935765, 0.00035325619670203783, 1.621356865480007e-09, 25]
True 55 RIB
['ztf-11'

['dmd-6', 'M00655_2.00', 'NSM', 78, 71, 55.800238877276804, 1.2723959866220735, 2.3842228203364496e-05, 5.971939717340818e-06, 1.1246427882675374e-15, 39]
True 163 PHA
['dmd-6', 'M00655_2.00', 'PHA', 163, 132, 116.60819149995024, 1.1319959455854893, 0.003650550378741211, 5.015192942141974e-06, 5.508160634203187e-24, 63]
True 160 PHB
['dmd-6', 'M00655_2.00', 'PHB', 160, 129, 114.4620284662088, 1.1270113043478263, 0.005432720716825985, 1.075271863757771e-05, 1.2546786296295223e-22, 59]
True 58 RIG
['dmd-6', 'M00655_2.00', 'RIG', 58, 51, 41.4924853190007, 1.2291382308845575, 0.0025089208262103284, 3.688425992663163e-05, 1.5953825916807262e-11, 27]
True 49 RIH
['dmd-6', 'M00655_2.00', 'RIH', 49, 41, 35.05399621777645, 1.1696241348713399, 0.03708849713271203, 0.02231953387073328, 5.834723765949796e-07, 20]
True 34 RIR
['dmd-6', 'M00655_2.00', 'RIR', 34, 29, 24.323181049069373, 1.1922782608695652, 0.04990702055922679, 0.012598933164847342, 7.682523910031353e-07, 13]
dmd-9 M00661_2.00
True 96

['nhr-6', 'M01298_2.00', 'PVW', 40, 26, 17.18722006569125, 1.51275191104934, 0.004071508867164515, 0.5310935747214449, 0.00021490524152340424, 8]
True 53 RID
['nhr-6', 'M01298_2.00', 'RID', 53, 32, 22.773066587040905, 1.4051686837033057, 0.007937388383199247, 0.022134238224184356, 5.932835304349066e-06, 14]
nhr-67 M02408_2.00
True 114 ADF
['nhr-67', 'M02408_2.00', 'ADF', 114, 54, 34.39743206927441, 1.5698846324123026, 8.117999969881078e-05, 0.0026863181370274086, 2.37779587611034e-09, 20]
True 50 AIN
['nhr-67', 'M02408_2.00', 'AIN', 50, 26, 15.086593012839655, 1.7233844631370607, 0.0010117600457412797, 0.007395087353507392, 1.2469885447321917e-05, 13]
True 169 ASE
['nhr-67', 'M02408_2.00', 'ASE', 169, 89, 50.992684383398036, 1.7453483980336642, 7.90990621549825e-10, 0.0008628649530833883, 9.227698705311929e-15, 41]
True 101 ASG
['nhr-67', 'M02408_2.00', 'ASG', 101, 49, 30.474917885936097, 1.6078796400174409, 8.051401983855481e-05, 0.012659854117328511, 3.955427662009166e-10, 21]
True 1

True 78 DA
['tra-1', 'DZ2013_TRA-1', 'DA', 78, 43, 18.86533293520454, 2.2793130737575185, 4.307092999186993e-09, 0.893346556320022, 1.2792548345979175e-07, 14]
True 40 PVW
['tra-1', 'DZ2013_TRA-1', 'PVW', 40, 17, 9.674529710361302, 1.7571913580246914, 0.008165195968007546, 0.44184766393951225, 0.002350133633624767, 4]
F52B5.7 M00625_2.00
True 38 PDB
['F52B5.7', 'M00625_2.00', 'PDB', 38, 15, 8.228227331541754, 1.8229928993939686, 0.009705221702180382, 0.015810779771671505, 4.526267719522638e-05, 9]
daf-12 M00685_2.00
True 43 PVN
['daf-12', 'M00685_2.00', 'PVN', 43, 27, 19.37931720911715, 1.3932379406689128, 0.01456561864497197, 0.0019255994917062313, 8.394437577316704e-07, 16]
fkh-7 M00792_2.00
True 153 PVT
['fkh-7', 'M00792_2.00', 'PVT', 153, 85, 69.06852791878173, 1.2306618160438025, 0.006037407175294111, 0.10116120590476077, 6.320580223867084e-12, 38]
ceh-33 M00563_2.00
True 40 MI
['ceh-33', 'M00563_2.00', 'MI', 40, 17, 9.76609933313427, 1.7407154504688136, 0.00900708767303243, 0.802

In [122]:
s.to_csv("../output/110419c_single_factor_analysis_data_DE_NB_sc.csv",sep=',')
b.to_csv("../output/110419c_single_factor_analysis_data_DE_NB_ba.csv",sep=',')